In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

## Generate zipcode employment/income map

In [2]:
from csv import DictReader

data_key = 'Percent; EMPLOYMENT STATUS - Civilian labor force - Percent Unemployed'
zipcode_key = 'Id2'

unemployment_results = {}
missing_data = []

with open(os.path.join(settings.DATA_DIR, 'ACS_14_5YR_DP03_with_ann.csv')) as fh:
    reader = DictReader(fh)
    for row in reader:
        try:
            unemployment_results[row[zipcode_key]] = float(row[data_key])
        except:
            missing_data.append(row[zipcode_key])
            continue

print 'Found %d zipcodes with missing data' % len(missing_data)

IOError: [Errno 2] No such file or directory: '/home/aepton/code/campfin/data/ACS_14_5YR_DP03_with_ann.csv'

In [3]:
for zc in sorted(unemployment_results, key=lambda k: unemployment_results[k], reverse=True)[:10]:
    print zc, unemployment_results[zc]

for zc in unemployment_results:
    if zc[:5] in zipcode_funding:
        zipcode_funding[zc[:5]]['unemployment'] = float(unemployment_results[zc])

In [4]:
from csv import DictWriter

with open(os.path.join(settings.DATA_DIR, 'acs_14_5_year_zipcode_unemployment.csv'), 'w+') as fh:
    writer = DictWriter(fh, ['zipcode', 'unemployment'])
    writer.writeheader()
    for zipcode in unemployment_results:
        writer.writerow({'zipcode': zipcode, 'unemployment': unemployment_results[zipcode]})

IOError: [Errno 2] No such file or directory: '/home/aepton/code/campfin/data/acs_14_5_year_zipcode_unemployment.csv'

## Median age and racial breakdown

In [5]:
# Just curious. What zipcodes have highest disproportion of men/women?
age_data_key = 'Estimate; SEX AND AGE - Median age (years)'
asian_data_key = 'Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Asian alone'
black_data_key = 'Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Black or African American alone'
hispanic_data_key = 'Percent; HISPANIC OR LATINO AND RACE - Total population - Hispanic or Latino (of any race)'
white_data_key = 'Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - White alone'
zipcode_key = 'Id2'

race_results = {}
missing_data = []

with open(os.path.join(settings.DATA_DIR, 'ACS_14_5YR_DP05_with_ann.csv')) as fh:
    reader = DictReader(fh)
    for row in reader:
        try:
            zipcode_funding[row[zipcode_key]]['median_age'] = float(row[age_data_key])
            zipcode_funding[row[zipcode_key]]['percent_asian'] = float(row[asian_data_key])
            zipcode_funding[row[zipcode_key]]['percent_black'] = float(row[black_data_key])
            zipcode_funding[row[zipcode_key]]['percent_hispanic'] = float(row[hispanic_data_key])
            zipcode_funding[row[zipcode_key]]['percent_white'] = float(row[white_data_key])
        except:
            missing_data.append(row[zipcode_key])
            continue

print 'Found %d zipcodes with missing data' % len(missing_data)

IOError: [Errno 2] No such file or directory: '/home/aepton/code/campfin/data/ACS_14_5YR_DP05_with_ann.csv'

## Save combined funding and demography data to CSV

In [6]:
from csv import DictWriter

paths = [
    os.path.join(settings.DATA_DIR, 'combined_funding_zipcode_demography.csv'),
    '/home/aepton/code/dataviz/732/combined_funding_demography.csv',
    '/home/aepton/code/dataviz/init_maps/combined_funding_demography.csv',
]

level_one = [
    'zipcode', 'unemployment', 'percent_white', 'percent_black', 'percent_asian', 'percent_hispanic', 'median_age']
level_two = []

for measure in measures:
    for side in ['yes', 'no']:
        level_two.append('%s: %s' % (measure, side))

for path in paths:
    with open(path, 'w+') as fh:
        writer = DictWriter(fh, level_one + level_two)
        writer.writeheader()
        for zipcode in zipcode_funding:
            if not zipcode:
                continue
            row = {
                'zipcode': zipcode
            }
            for column in level_one:
                if column == 'zipcode':
                    continue
                row[column] = zipcode_funding[zipcode][column]
            for measure in measures:
                for side in ['yes', 'no']:
                    row['%s: %s' % (measure, side)] = zipcode_funding[zipcode][measure][side]
            writer.writerow(row)

NameError: name 'measures' is not defined